In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import time
# from google.colab import drive
import os
import gc
from scipy.spatial.distance import cdist, pdist, squareform
# def mount():
#   drive.mount('/content/drive')
#   os.chdir("/content/drive/My Drive/SLAss2")
# mount()

In [2]:
# train = pd.read_csv("dtrain123.dat", delimiter = "  ", header = None)
test = pd.read_csv("data/dtest123.dat", delimiter = "  ", header = None)
train = pd.read_csv("data/zipcombo.dat", delimiter = " ", header = None)
train.shape
minitrain = pd.read_csv("data/dtrain123.dat", delimiter = "  ", header = None)
train = np.array(train)
xtrain = train[:,1:257]
ytrain = train[:,0]
one = np.array(minitrain)
xone = one[:, 1:257]
yone = one[:,0]
test = np.array(test)
xtest = test[:, 1:]
ytest = test[:,0]
def split_data(fullK, jtrain, jtest, y):
    Ktrain = fullK[jtrain][:,jtrain].copy()
    Ktest =  fullK[jtrain][:,jtest].copy()
    ytrain = y[jtrain]
    ytest = y[jtest]
    return Ktrain, ytrain, Ktest, ytest


<ipython-input-2-ad997d494748>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test = pd.read_csv("dtest123.dat", delimiter = "  ", header = None)
<ipython-input-2-ad997d494748>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  minitrain = pd.read_csv("dtrain123.dat", delimiter = "  ", header = None)


In [3]:
class kernel_perceptron:
    def __init__(self, N):
        self.N = N
        self.alpha = np.zeros((self.N, 1))
        self.accs = []
        self.w = np.zeros((N, 1))
    def train(self, K, y, d = 2, epochs = 1, alt = True):
        if alt == False:
            if len(self.alpha) != len(K):
                self.N = len(K)
                self.alpha = np.zeros((len(K), 1))
            for epoch in range(epochs):
                mistakes = 0
                yhats = np.zeros(self.N)
                js = np.arange(self.N)
                np.random.shuffle(js)
                for i in js:
                    yhat = np.sign(np.sign((K[:,i].dot(self.alpha))))
                    if yhat != y[i]:
                        self.alpha[i] += y[i]
                        mistakes += 1
                    # else:
                    #     self.alpha[js[i]] = 0
                    yhats[i] = yhat
                self.accs.append(1 - mistakes/self.N)
            return yhats
        else:
            js = np.arange(self.N)
            np.random.shuffle(js)
            yhat = np.sign(K.dot(self.alpha)).flatten()
            for i in range(self.N):
                if yhat[js[i]] != y[js[i]]:
                    self.alpha[js[i]] += y[js[i]]
                    yhat[js[i:]] = np.sign(K[js[i:], :].dot(self.alpha)).flatten()

    
    def predict(self, Ktest, ytest):
        return Ktest.T.dot(self.alpha).flatten()
                
def K(p, q, d = 2):
    return (p.T.dot(q))**d

# def GaussK(p, q, c):
#     print(np.sum((p-q)**2))
#     return np.exp(-c * np.sum((p-q)**2))

class onevsall:
    def __init__(self, xtrain, ytrain, xtest = None, ytest = None, kernel = K, d = 2, ks = None):
        self.N = len(xtrain)

        self.models = []
        self.accs = []
        self.testaccs = []
        self.d = d
        self.x, self.xtest = xtrain, xtest
        self.y, self.ytest = ytrain, ytest
        if ks == None:
            self.K = np.zeros((self.N, self.N))
            for i in range(self.N):
                for j in range(i, self.N):
                    # print(self.x[i], self.x[j], d)
                    self.K[i,j] = K(self.x[i], self.x[j], d)
            self.K = self.K + self.K.T - np.diag(self.K.diagonal())
            self.Ktest = np.zeros((self.N, len(self.xtest)))
            for i in range(self.N):
                for j in range(len(self.xtest)):
                    self.Ktest[i,j] = K(self.x[i], self.xtest[j], d)
        else:
            self.N = len(ks[0])
            self.K, self.Ktest = ks
    
    def train(self, print_accs = True, alt = False):
        self.vals = range(int(max(self.y)+1)) # final yval doesn't need model
        if len(self.models) == 0:
            self.ytemps = []
            for val in self.vals:
                ytemp = self.y.copy()
                ytemp[ytemp != val] = -1
                ytemp[ytemp == val] = 1
                self.add_model(ytemp)
                self.ytemps.append(ytemp.copy())
        for val in self.vals:
            self.models[val].train(self.K, self.ytemps[val], epochs = 1, alt = alt)

        if print_accs:
            yhat = self.predict(self.K, self.y)
            acc = np.mean(yhat.reshape(len(yhat), 1) == self.y.reshape(len(yhat), 1))
            self.accs.append(acc)            
            yhattest = self.predict(self.Ktest, self.ytest)
            testacc = np.mean(yhattest.reshape(len(yhattest), 1) == self.ytest.reshape(len(yhattest), 1))
            self.testaccs.append(testacc)
            print(f"Training Accuracy: {acc}. Testing Accuracy: {testacc}")
            return acc, testacc

    def add_model(self, ytrain):
        model = kernel_perceptron(self.N)
        # model.train(self.K, ytrain, epochs = epochs, d = self.d)
        self.models.append(model)
    
    def predict(self, k, ytest, full = False):
        preds = np.zeros((k.shape[1], len(self.models)))
        yhat = np.zeros(k.shape[1])
        for i, model in enumerate(self.models):
            preds[:,i] = model.predict(k, ytest)
        if full == True:
            return preds
        return preds.argmax(axis = 1)




In [4]:
def RBF(x, c):
    edist = squareform(pdist(x, metric = "euclidean"))**2
    edist /= edist.max()
    return np.exp(-c * edist)

# 1.1

In [ ]:
cs = [3, 5, 6, 6.5, 7, 7.5, 8, 9]
metrics = np.zeros((20, len(cs), 2))
ntrain = int(0.8*len(xtrain))

for ic, c in enumerate(cs):
    Kfull = RBF(xtrain, c)
    js = np.arange(len(xtrain))
    for i in range(20):
        np.random.seed(i)
        np.random.shuffle(js)
        jtrain = js[:ntrain]
        jtest = js[ntrain:]
        ktrain = Kfull[jtrain][:,jtrain]
        ktest = Kfull[jtrain][:,jtest]
        xt =  xtrain[jtrain]
        yt = ytrain[jtrain]
        ytest = ytrain[jtest]
        model = onevsall(xt, yt, None, ytest, ks = (ktrain, ktest))
        for epoch in range(10):
            model.train(False, False)
        metrics[i, ic] = model.train(True, False)
        print(c, ic, metrics[i, c])
    np.save("GaussNew1.1", metrics)
    ic += 1

Training Accuracy: 0.9995966657703684. Testing Accuracy: 0.9736559139784946
3 0 [0. 0.]
Training Accuracy: 0.9998655552567894. Testing Accuracy: 0.978494623655914
3 0 [0. 0.]
Training Accuracy: 0.9997311105135789. Testing Accuracy: 0.9682795698924731
3 0 [0. 0.]


# 1.2

In [23]:
cs = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]
ntrain = int(0.8*len(xtrain))
metrics = np.zeros((len(cs), 20, 2))
nfold = int(0.8 * ntrain)
nfoldtest = int(ntrain - nfold)
cstars = np.zeros(20)
testerr = np.zeros(20)
confusion_matrix = np.zeros((10, 10))
cvfullnew = np.zeros((len(cs), 20, 5, 2))
print("Start")
start = time.time()
for i in range(20):
    np.random.seed(i)
    js = np.arange(len(xtrain))
    np.random.shuffle(js)
    jtrain = js[:ntrain]
    jtest = js[ntrain:]
    # jtrain = np.arange(0, 5000)
    # rng = np.random.default_rng()
    bestacc = 0
    cstar = None
    for ic, c in enumerate(cs):

        # split into 80/20
        fullKd = RBF(xtrain, c)
        Ktrain, ytraini, Ktest, ytesti = split2(fullKd, jtrain, jtest, ytrain, ntrain)
        # five-fold cross validation
        testacc = 0
        for k in range(5):
            nsplit = int(nfold * k/5)
            ktrain1 = np.arange(0, nsplit+1)
            ktest = np.arange(nsplit+1, nsplit + nfoldtest-1)
            ktrain2 = np.arange(nsplit + nfoldtest-1, nfold)
            ktrain = np.hstack([ktrain1, ktrain2]).flatten()
            Kfoldtrain, yfoldtraini, Kfoldtest, yfoldtesti = split2(Ktrain, ktrain, ktest, ytraini, nfold)
            model = onevsall(yfoldtraini, yfoldtraini, None, yfoldtesti, ks = (Kfoldtrain, Kfoldtest))
            for epoch in range(5):
                model.train(False, False)    
            yhat = model.predict(Kfoldtest, yfoldtesti)
            testacc += 0.2 * np.mean(yhat.flatten() == yfoldtesti.flatten())
            cvfullnew[ic, i, k] = [np.mean(model.predict(Kfoldtrain, yfoldtraini).flatten() == yfoldtraini.flatten()),
                                    np.mean(yhat.flatten() == yfoldtesti.flatten())]
        print(f"{i} run : {c}: {testacc}, {bestacc}, {cstar}")
        np.save(open("fullGaussCVFINAL", "wb"), cvfullnew)
        
        
        if testacc > bestacc:
            bestacc = testacc
            cstar = c
#             yhat = model.predict(Ktest)
#             testerr[i] = 1 - np.mean(yhat.flatten() == ytesti.flatten())
    #train final model for run
    fullKd = RBF(xtrain, cstar)
    Ktrainfinal, ytraini, Ktestfinal, ytesti = split2(fullKd, jtrain, jtest, ytrain, ntrain)
    finalmodel = onevsall(ytrain, ytraini, None, ytesti, ks = (Ktrainfinal, Ktestfinal))
    for epoch in range(5):
        finalmodel.train(False, False)
    finalyhat = finalmodel.predict(Ktestfinal, ytesti)
    cstars[i] = cstar
    testerr[i] = 1 - (finalmodel.predict(finalmodel.Ktest, finalmodel.ytest).flatten() == finalmodel.ytest.flatten()).mean()
    print(f"{i} runs finished. Dstar: {cstar}, Testerr: {testerr[i]}, Time: {time.time() -start}")
    np.save(open("GaussCVNew", "wb"), testerr)
    np.save(open("cstarsGaussNew", "wb"), cstars)

Start
0 run : 6.5: 0.966621803499327, 0, None
0 run : 7: 0.967563930013459, 0.966621803499327, 6.5
0 run : 7.5: 0.9681022880215343, 0.967563930013459, 7
0 run : 8: 0.9687752355316286, 0.9681022880215343, 7.5
0 run : 9: 0.9694481830417228, 0.9687752355316286, 8
0 run : 10: 0.9695827725437417, 0.9694481830417228, 9
0 run : 11: 0.9679676985195156, 0.9695827725437417, 10
0 run : 12: 0.9644683714670256, 0.9695827725437417, 10
0 runs finished. Dstar: 10, Testerr: 0.02580645161290318, Time: 474.79002141952515
1 run : 6.5: 0.9652759084791386, 0, None
1 run : 7: 0.9667563930013459, 0.9652759084791386, 6.5
1 run : 7.5: 0.9664872139973082, 0.9667563930013459, 7
1 run : 8: 0.9655450874831762, 0.9667563930013459, 7
1 run : 9: 0.9672947510094213, 0.9667563930013459, 7
1 run : 10: 0.966756393001346, 0.9672947510094213, 9
1 run : 11: 0.9683714670255721, 0.9672947510094213, 9
1 run : 12: 0.9631224764468371, 0.9683714670255721, 11
1 runs finished. Dstar: 11, Testerr: 0.02311827956989243, Time: 944.10605

16 run : 7.5: 0.966621803499327, 0.9639300134589504, 6.5
16 run : 8: 0.9668909825033647, 0.966621803499327, 7.5
16 run : 9: 0.9675639300134591, 0.9668909825033647, 8
16 run : 10: 0.9640646029609691, 0.9675639300134591, 9
16 run : 11: 0.9631224764468371, 0.9675639300134591, 9
16 run : 12: 0.9656796769851951, 0.9675639300134591, 9
16 runs finished. Dstar: 9, Testerr: 0.02580645161290318, Time: 10201.839571237564
17 run : 6.5: 0.9664872139973083, 0, None
17 run : 7: 0.9658142664872142, 0.9664872139973083, 6.5
17 run : 7.5: 0.9648721399730821, 0.9664872139973083, 6.5
17 run : 8: 0.9668909825033648, 0.9664872139973083, 6.5
17 run : 9: 0.97092866756393, 0.9668909825033648, 8
17 run : 10: 0.9683714670255721, 0.97092866756393, 9
17 run : 11: 0.9662180349932706, 0.97092866756393, 9
17 run : 12: 0.968371467025572, 0.97092866756393, 9
17 runs finished. Dstar: 9, Testerr: 0.02741935483870972, Time: 10659.737754106522
18 run : 6.5: 0.9635262449528936, 0, None
18 run : 7: 0.9621803499327053, 0.96352